In [5]:
from matplotlib import pyplot as plt
from IPython.core.display import display
from gurobipy import Model, GRB, quicksum
from pandas import DataFrame
from numpy import linspace

In [6]:
Pa = ["NG Boiler","NG CHP", "B Boiler", "B CHP", "Solar PV", "Wind Turbine"] #Process A
Pb = ["CAR, RwHR, RwoHR, FL, LED, Ba, Bb, Ha, Hb"] #Process B
E = ["Natural Gas", "Biomass",] #Primary Energy Sources
D = ["Space Heating, Ventilation, Refigeration, Lighting, Bakery"] #Demands

In [7]:
ppa = DataFrame({"NG Boiler": [0, 0.9, 100, 10**6, 7000, 200, 10], 
                 "NG CHP": [0.35, 0.55, 800, 10**6, 7000, 500, 15], 
                 "B Boiler": [0, 0.85, 100,10**6, 7000, 250, 15], 
                 "B CHP": [0.33, 0.5, 1000, 10**6, 7000, 2000, 30], 
                 "Solar PV": [0, 0, 10, 20, 800, 2000, 500], 
                 "Wind Turbine": [0, 0, 10, 30, 1750, 2000, 1200]}, 
                index=["Ne", "Nh", "CapL", "CapU", "T", "Inv", "OM"]).transpose()

get_ne = lambda j: ppa["Ne"][j]
get_Nh = lambda j: ppa["Nh"][j]
get_CapL = lambda j: ppa["CapL"][j]
get_CapU = lambda j: ppa["CapU"][j]
get_T = lambda j: ppa["T"][j]
get_Inv = lambda j: ppa["Inv"][j]
get_OM = lambda j: ppa ["OM"][j]

In [8]:
ppb = DataFrame({"CAR": [6, 50, 3], 
                 "RwHR": [3, 100, 5],
                 "RwHR": [2, 100, 5],
                 "RwoHR": [3, 70, 4], 
                 "FL": [0.2, 5, 0.5], 
                 "LED": [0.8, 10, 1], 
                 "Ba": [0.7, 30, 3], 
                "Bb": [0.75, 40, 4],
                "Ha": [0.85, 30, 3],
                "Hb": [0.9, 40, 4]},
                index=["Efficiency", "Inv", "OM"]).transpose()

get_efficiency = lambda j: ppb["Efficiency"][j]
get_inv = lambda j: ppb["Inv"][j]
get_om = lambda j: ppb["OM"][j]

In [9]:
mp = DataFrame({"Natural Gas": [8.89, 56342], "Biomass": [9.72, 100970]}, 
               index=["Price", "Co2"]).transpose()

get_price = lambda i: mp["Price"][i]
get_co2 = lambda i: mp["Co2"][i]


In [10]:
m = Model()

Using license file /Users/zacblackburn/gurobi.lic


GurobiError: License expired 2020-11-30

In [ ]:
#variables
NG=m.addVar(vtype=GRB.CONTINUOUS)
NG_1=m.addVar(vtype=GRB.CONTINUOUS)
NG_2=m.addVar(vtype=GRB.CONTINUOUS)
B=m.addVar(vtype=GRB.CONTINUOUS)
B_1=m.addVar(vtype=GRB.CONTINUOUS)
B_2=m.addVar(vtype=GRB.CONTINUOUS)

CNGB=m.addVar(vtype=GRB.CONTINUOUS)
CBB=m.addVar(vtype=GRB.CONTINUOUS)

CNGCHPH=m.addVar(vtype=GRB.CONTINUOUS)
CBCHPH=m.addVar(vtype=GRB.CONTINUOUS)

CNGCHPE=m.addVar(vtype=GRB.CONTINUOUS)
CBCHPE=m.addVar(vtype=GRB.CONTINUOUS)

W=m.addVar(vtype=GRB.CONTINUOUS)
S=m.addVar(vtype=GRB.CONTINUOUS)
CW=m.addVar(vtype=GRB.CONTINUOUS)
CS=m.addVar(vtype=GRB.CONTINUOUS)

GRID=m.addVar(vtype=GRB.CONTINUOUS)
CGRID=m.addVar(vtype=GRB.CONTINUOUS)



Ha=m.addVar(vtype=GRB.CONTINUOUS)
Hb=m.addVar(vtype=GRB.CONTINUOUS)
CAR=m.addVar(vtype=GRB.CONTINUOUS)
RwHR=m.addVar(vtype=GRB.CONTINUOUS)
RwoHR=m.addVar(vtype=GRB.CONTINUOUS)
FL=m.addVar(vtype=GRB.CONTINUOUS)
LED=m.addVar(vtype=GRB.CONTINUOUS)
Ba=m.addVar(vtype=GRB.CONTINUOUS)
Bb=m.addVar(vtype=GRB.CONTINUOUS)
RwHRsh=m.addVar(vtype=GRB.CONTINUOUS)
RwHRr=m.addVar(vtype=GRB.CONTINUOUS)

SHT=m.addVar(vtype=GRB.CONTINUOUS)
VT=m.addVar(vtype=GRB.CONTINUOUS)
RT=m.addVar(vtype=GRB.CONTINUOUS)
LT=m.addVar(vtype=GRB.CONTINUOUS)
BT=m.addVar(vtype=GRB.CONTINUOUS)

HT=m.addVar(vtype=GRB.CONTINUOUS)
ET=m.addVar(vtype=GRB.CONTINUOUS)

NGT=m.addVar(vtype=GRB.CONTINUOUS)
BMT=m.addVar(vtype=GRB.CONTINUOUS)
CNGCHP=m.addVar(vtype=GRB.CONTINUOUS)
CBCHP=m.addVar(vtype=GRB.CONTINUOUS)

In [ ]:
#Binary Var
Y1=m.addVar(vtype=GRB.BINARY)
Y2=m.addVar(vtype=GRB.BINARY)
Y3=m.addVar(vtype=GRB.BINARY)
Y4=m.addVar(vtype=GRB.BINARY)
Y5=m.addVar(vtype=GRB.BINARY)
Y6=m.addVar(vtype=GRB.BINARY)
Y7=m.addVar(vtype=GRB.BINARY)
Y8=m.addVar(vtype=GRB.BINARY)
Y9=m.addVar(vtype=GRB.BINARY)
Y10=m.addVar(vtype=GRB.BINARY)
Y11=m.addVar(vtype=GRB.BINARY)
Y12=m.addVar(vtype=GRB.BINARY)
Y13=m.addVar(vtype=GRB.BINARY)
Y14=m.addVar(vtype=GRB.BINARY)
Y15=m.addVar(vtype=GRB.BINARY)

In [ ]:
#CONSTRAINTS 

m.addConstr(SHT >= 100),
m.addConstr(VT >= 100),
m.addConstr(RT >=200),
m.addConstr(LT >= 100),
m.addConstr(BT >= 100),


m.addConstr(HT == 0.9*NG_1 + 0.85*B_1 + 0.55*NG_2 + 0.5*B_2),
m.addConstr(HT == Ha + Hb),
m.addConstr(0 <= 0.9*NG_1 + 0.85*B_1 + 0.55*NG_2 + 0.5*B_2 - Ha - Hb),
m.addConstr(ET == GRID + S + 0.35*NG_2 + 0.33*B_2),
m.addConstr(ET == CAR + RT + LT + BT),
m.addConstr(0 <= GRID + S + 0.35*NG_2 + 0.33*B_2 - CAR + RT + LT + BT),
m.addConstr(NGT == NG_1 + NG_2),
m.addConstr(BMT == B_1 + B_2),
m.addConstr(CNGCHP == CNGCHPE + CNGCHPH),
m.addConstr(CBCHP == CBCHPH + CBCHPE),

m.addConstr(0.9*NG_1 == CNGB * 7000 * (1/8760))
m.addConstr(0.85*B_1 == CBB * 7000 * (1/8760))
m.addConstr(0.55*NG_2 == CNGCHPH * 7000 * (1/8760))
m.addConstr(0.5*B_2 == CBCHPH * 7000 * (1/8760))
m.addConstr(0.35*NG_2 == CNGCHPE * 7000 * (1/8760))
m.addConstr(0.33*B_2 == CBCHPE * 7000 * (1/8760))


m.addConstr(3*RwHRr + 3*RwoHR >= 200)
m.addConstr(6*CAR >= 100)
m.addConstr(0.2*FL + 0.8*LED >= 100)
m.addConstr(2*RwHRsh + 0.85*Ha + 0.9*Hb >= 100)
m.addConstr(0.7*Ba + 0.75*Bb >= 100)


In [ ]:
m.addConstr(GRID <= 10**9*Y1)

m.addConstr(Y2 * 10 <= W)
m.addConstr(W <= 30 * Y2)

m.addConstr(Y3 * 10 <= S)
m.addConstr(S <= 20 * Y3)

m.addConstr(Y4 * 100 <= CNGB)
m.addConstr(CNGB <= 10**6 * Y4)

m.addConstr(Y5 * 100 <= CBB)
m.addConstr(CBB <= 10**6 * Y5)

m.addConstr(Y6 * 800 <= CNGCHP)
m.addConstr(CNGCHP <= 10**6 * Y6)

m.addConstr(Y7 * 1000 <= CBCHP)
m.addConstr(CBCHP <= 10**6 *Y7)

m.addConstr(Y1 + Y2 + Y3 + Y4 + Y5 + Y6 + Y7 <= 7)

m.addConstr(RwHR <= 10**9 * Y8)
m.addConstr(RwoHR <= 10**9 * Y9)
m.addConstr(FL <= 10**9 * Y10)
m.addConstr(LED <= 10**9 * Y11)
m.addConstr(Ha <= 10**9 * Y12)
m.addConstr(Hb <= 10**9 * Y13)
m.addConstr(Ba <= 10**9 * Y14) 
m.addConstr(Bb <= 10**9 * Y15)

In [ ]:
m.setObjective((0.203*8760*20*NGT+0.363*8760*20*BMT+0.322*8760*20*GRID),GRB.MINIMIZE)

In [ ]:
m.optimize()

In [ ]:
Y1, Y2, Y3, Y4, Y5, Y6, Y7, Y8, Y9, Y10, Y11, Y12, Y13, Y14, Y15

In [ ]:
GRID, CAR, Hb, Ha, Bb, Ba, LED, FL, RwoHR, CBCHP, CNGCHP, CBB, CNGB, RwHRsh, RwHRr